In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
%matplotlib inline

In [ ]:
import os, sys, inspect
# realpath() will make your script run, even if you symlink it :)
cmd_folder = os.path.realpath(os.path.abspath(
        os.path.join(os.path.split(inspect.getfile( inspect.currentframe() ))[0],
                     '..', '..', 'labeled-experiments', 'nn-classifiers')))
print 'Added {0} to path.'.format(cmd_folder)
if cmd_folder not in sys.path:
    sys.path.insert(0, cmd_folder)

In [ ]:
import labeling_network as lbln
from matplotlib import pyplot as plt
import matplotlib.colors
import theano
import theano.tensor as T
import numpy as np
from labeling_network import FullyConnectedLayer, ConvPoolLayer

import time
import datetime

In [ ]:
dataPath = '../../../factory-robot-data/imgs_2015-10-18-2/'

In [ ]:
n_direction_sensors = 9
n_classes = 2
rng = np.random.RandomState()

In [ ]:
load_time_start = time.time()
train_data, valid_data, test_data = lbln.load_data_shared(
                     dataPath,
                     'capture',
                     dataPath + 'labels.dat',
                     n_train=16000,
                     n_validation=2000,
                     n_test=100,
                     label_mask=-1,
                     n_direction_sensors=n_direction_sensors,
                     n_classes=n_classes,
                     shuffle_rng=rng,
                     hsv='hsv')

# reverse target and label
train_data = train_data[1], train_data[0]
valid_data = valid_data[1], valid_data[0]
test_data = test_data[1], test_data[0]


print 'Loading data took {0:.5} seconds'.format(time.time() - load_time_start)

In [ ]:
mini_batch_size = 20

### Train the Network. 

In [ ]:
network = lbln.Network([
        
#         ConvPoolLayer(image_shape=(mini_batch_size, 3, 64, 64), 
#                       filter_shape=(20, 3, 7, 7), 
#                       poolsize=(2, 2),
#                       activation_fn=lbln.ReLU),
#         FullyConnectedLayer(n_in=29*29*20, n_out=128, p_dropout=0.5),
#         FullyConnectedLayer(n_in=128, n_out=5)
        
        
        FullyConnectedLayer(n_in=n_classes*n_direction_sensors, n_out=2048),
#         FullyConnectedLayer(n_in=128, n_out=2048),
        FullyConnectedLayer(n_in=2048, n_out=64*64*3)
        
#         FullyConnectedLayer(n_in=25, n_out=400, p_dropout=0.0),
#         FullyConnectedLayer(n_in=400, n_out=64*64*3)
        
    ], mini_batch_size)

In [ ]:
network.SGD(train_data, 
            1000, 
            mini_batch_size, 
            10.0, 
            valid_data, 
            test_data,
            'trained-networks/decoders/d_' + datetime.datetime.now().strftime('%Y%m%d%H%M%S'),
#             rmsprop=(0.001, 0.9, 1e-6, 1.0),
            adam=(0.01, 0.1, 0.001, 1e-8)
#             learning_curve_file_name='decoder_learning_curve_bigdata'
           )

#### Save network weights.

In [ ]:
# network.save_as_file('fire_classifier')

In [ ]:
# network = network.load_from_file('best_encoder_bigdata', 10)

In [ ]:
import pylab
pylab.rcParams['figure.figsize'] = (6.0, 6.0)

In [ ]:
train_data_xs = train_data[0].get_value()
train_data_ts = train_data[1].get_value()

test_data_xs = test_data[0].get_value()
test_data_ts = test_data[1].get_value()

valid_data_xs = valid_data[0].get_value()
valid_data_ts = valid_data[1].get_value()


### Determine all Losses on test images.

In [ ]:
def show_image(img, hsv=None):
    if hsv is None:
        plt.imshow(img.reshape(64, 64, 3), interpolation='nearest')
        plt.show()
    else:
        plt.imshow(matplotlib.colors.hsv_to_rgb(img.reshape(64, 64, 3)), interpolation='nearest')
        plt.show()

In [ ]:
def print_classification(index, n_classes, n_directions):
#     print 'Target:    ', classifications[index][1]
#     print 'Prediction:', classifications[index][2]
    print 'Loss:      ', classifications[index][3]
    show_image(classifications[index][1], hsv='hsv')
    show_image(classifications[index][2], hsv='hsv')
    
    
    
    fig = plt.figure(figsize=(12,6))
    ax_left = fig.add_subplot(111)
    ax_left.set_ylim(0,1)
    bar_width = 0.1
    chart_colors = ['#ff2222', '#3399bb', '#55aa22', '#bbee33', '#881144']
    for i in xrange(n_classes):
        ax_left.bar(np.arange(n_directions)-(n_directions/2) + (i - n_directions/2.0 + 0.5)*bar_width,
               classifications[index][0][i*n_directions: (i+1)*n_directions],
               bar_width,
               color=chart_colors[i],
               align='center')
    plt.show()
    print '---------'
        


# Determine all errors on the test images
classifications = []
for x, t in zip(valid_data_xs, valid_data_ts):
    prediction = network.get_single_output(x)
    error = np.mean((t - prediction) ** 2)
    classifications.append((x, t, prediction, error))

print '--- Worst predictions: ---'
classifications = sorted(classifications, key = lambda (x, t, p, e): e, reverse=True)
print_classification(0, n_classes, n_direction_sensors)
print_classification(1, n_classes, n_direction_sensors)

print '\n\n'
print '--- Median prediction ---'
n_displayed_median = 8
for i in xrange(-n_displayed_median/2, n_displayed_median/2):
    print_classification(len(classifications)/2 + i, n_classes, n_direction_sensors)
    
fig = plt.figure()
ax = fig.add_subplot(111)
ax.bar(np.arange(len(classifications)), np.asarray(classifications)[:, 3])
plt.show()